In [16]:
import pandas as pd
import spacy
import en_core_web_sm
from collections import Counter

In [17]:
df = pd.read_csv("./downloads/reviews100.csv")
df

,stars,comment
0,4.0 out of 5 stars,I purchased this bad boy for my wife’s home of...
1,4.0 out of 5 stars,This is a quite nice machine. Very basic yet b...
2,4.0 out of 5 stars,PROS: I want to cover first the high-quality a...
3,4.0 out of 5 stars,Last it lasted 6 years! And this one cost $4 l...
4,5.0 out of 5 stars,"Update: After over a year, the shredder is sti..."
...,...,...
95,4.0 out of 5 stars,Very good value for this Amazon Basic cross cu...
96,5.0 out of 5 stars,This is a great machine at an even better pric...
97,1.0 out of 5 stars,"So I decided I wanted to get a paper shredder,..."
98,1.0 out of 5 stars,I have now been through 3 of these shredders. ...


In [18]:
df['comment'] = df['comment'].apply(lambda x: x.strip())
df = df.sort_values(['stars'], ascending=False).iloc[0:30] # get 30 top reviews 
df

,stars,comment
99,5.0 out of 5 stars,If I could give this six stars I would. What a...
32,5.0 out of 5 stars,This shredder was purchased to replace a Fello...
77,5.0 out of 5 stars,I purchased this shredder despite some bad rev...
22,5.0 out of 5 stars,The shredder does exactly what it says it will...
67,5.0 out of 5 stars,I bought this shredder last week after looking...
27,5.0 out of 5 stars,It does the job and does it fast! We own a not...
64,5.0 out of 5 stars,Pretty thrilled with this shredder right now !...
57,5.0 out of 5 stars,Cross-cut is key and Amazon has made this prod...
55,5.0 out of 5 stars,"I just got the shredder, and have been using i..."
18,5.0 out of 5 stars,This works great! I don't typically test it wi...


In [19]:
# nlp = spacy.load("en_core_web_sm") # buggy on ipynb
nlp = en_core_web_sm.load() # for ipynb
noun_adj_pairs = {}
# print(df['comment'])
for comment in enumerate(df['comment']):
    # print(comment[1])
    doc = nlp(str(comment[1])) # change tuple to str for nlp
    doc_len = len(doc)
    for i, token in enumerate(doc): #token has TEXT, LEMMA, POS, TAG, DEP, SHAPE, ALPHA, STOP
        if token.pos_ not in ('NOUN', 'PROPN'):
            continue
        for j in range(i+1, doc_len): # check for pairs after identified noun/pnoun
            if doc[j].pos_ == 'ADJ':
                pair = (str(token), str(doc[j])) # change to str. ele in dict keys has to be hashable even inside tuples
                # check if already exist in dictionary
                if pair in noun_adj_pairs:
                    noun_adj_pairs[pair] += 1
                else:
                    noun_adj_pairs[pair] = 1
top_10_noun_adj_pairs = dict(Counter(noun_adj_pairs).most_common(5))
top_10_noun_adj_pairs
# top_10_noun_adj_pairs = {key: val for key, val in sorted(noun_adj_pairs.items(), key=lambda item: item[1])}
# print(noun_adj_pairs.keys())
# noun_adj_pairs = dict(sorted(noun_adj_pairs.items()))
# print(noun_adj_pairs)

{('shredder', 'small'): 36,
 ('shredder', 'little'): 34,
 ('shredder', 'few'): 32,
 ('shredder', 'full'): 21,
 ('shredder', 'sure'): 21}